<a href="https://colab.research.google.com/github/leoqzm/NER_Tasks/blob/main/N2C2_data_preprocessing(BratToBIO).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-processing procedure for 2018 n2c2 dataset

## functions 

In [ ]:
from pathlib import Path
import pickle as pkl
from collections import defaultdict, Counter
from itertools import permutations, combinations
from functools import reduce
import numpy as np
import os
import re

In [ ]:
def pkl_save(data, file):
    with open(file, "wb") as f:
        pkl.dump(data, f)

        
def pkl_load(file):
    with open(file, "rb") as f:
        data = pkl.load(f)
    return data


def load_text(ifn):
    with open(ifn, "r") as f:
        txt = f.read()
    return txt


def save_text(text, ofn):
    with open(ofn, "w") as f:
        f.write(text)

In [ ]:
import sys
!git clone https://github.com/uf-hobi-informatics-lab/NLPreprocessing
sys.path.append("/content/NLPreprocessing")
sys.path.append("/content/NLPreprocessing/text_process")
from annotation2BIO import pre_processing, read_annotation_brat, generate_BIO
MIMICIII_PATTERN = "\[\*\*|\*\*\]"
from sentence_tokenization import logger as l1
from annotation2BIO import logger as l2
l1.disabled = True
l2.disabled = True

Cloning into 'NLPreprocessing'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 99 (delta 54), reused 70 (delta 31), pack-reused 0
Unpacking objects: 100% (99/99), done.


In [ ]:
def create_entity_to_sent_mapping(nnsents, entities, idx2e):
    loc_ens = []
    
    ll = len(nnsents)
    mapping = defaultdict(list)
    for idx, each in enumerate(entities):
        en_label = idx2e[idx]
        en_s = each[2][0]
        en_e = each[2][1]
        new_en = []
        
        i = 0
        while i < ll and nnsents[i][1][0] < en_s:
            i += 1
        s_s = nnsents[i][1][0]
        s_e = nnsents[i][1][1]

        if en_s == s_s:
            mapping[en_label].append(i)

            while i < ll and s_e < en_e:
                i += 1
                s_e = nnsents[i][1][1]
            if s_e == en_e:
                 mapping[en_label].append(i)
            else:
                mapping[en_label].append(i)
                print("last index not match ", each)
        else:
            mapping[en_label].append(i)
            print("first index not match ", each)

            while i < ll and s_e < en_e:
                i += 1
                s_e = nnsents[i][1][1]
            if s_e == en_e:
                 mapping[en_label].append(i)
            else:
                mapping[en_label].append(i)
                print("last index not match ", each)
    return mapping


def get_permutated_relation_pairs(eid2idx):
    all_pairs = []
    all_ids = [k for k, v in eid2idx.items()]
    for e1, e2 in permutations(all_ids, 2):
        all_pairs.append((e1, e2))
    return all_pairs

In [ ]:
"""
procedure:

1. preprocess text into sentences
2. find each entity associated sentence idx
3. generate entity pairs as relation candidates
4. extract eneity associated sentences, locate entities in pair and insert special tags
5. save generated data

result output:

1. keep pos predicted relations
2. using map files to locate relation associated entities
3. output as brat
"""
def validate_rels(rels, valid):
    nrels = []
    for rel in rels:
        rtype = rel[0]
        if tuple(rtype) not in valid:
            print("invalid: ", rel)
            continue
        nrels.append(rel)
    return nrels


def check_tags(s1, s2):
    assert EN1_START in s1 and EN1_END in s1, f"tag error: {s1}"
    assert EN2_START in s2 and EN2_END in s2, f"tag error: {s2}"


def format_relen(en, rloc, nsents):
    if rloc == 1:
        spec1, spec2 = EN1_START, EN1_END
    else:
        spec1, spec2 = EN2_START, EN2_END
    sn1, tn1 = en[0][3]
    sn2, tn2 = en[-1][3]
    target_sent = nsents[sn1]
    target_sent = [each[0] for each in target_sent]
    ors =  " ".join(target_sent)
    
    if sn1 != sn2:
        print("[!!!Warning] The entity is not in the same sentence\n", en)
        tt = nsents[sn2]
        tt = [each[0] for each in tt]
        target_sent.insert(tn1, spec1)
        tt.insert(tn2+1, spec2)
        target_sent = target_sent + tt
#         print(target_sent)
    else:
        target_sent.insert(tn1, spec1)
        target_sent.insert(tn2+2, spec2)
    
    fs = " ".join(target_sent)
    
    return sn1, sn2, fs, ors


def gene_true_relations(rels, mappings, ens, e2i, nnsents, nsents, valid_comb, fid=None):
    true_pairs = set()
    pos_samples = []
    
    for rel in rels:
        rel_type = rel[0]
        enid1, enid2 = rel[1:]
        """
        [['100', (15443, 15446), (16473, 16476), (231, 4), 'B-Strength'], 
        ['mg', (15447, 15449), (16477, 16479), (231, 5), 'I-Strength']] 
        [['Metoprolol', (15422, 15432), (16452, 16462), (231, 2), 'B-Drug'], 
        ['Succinate', (15433, 15442), (16463, 16472), (231, 3), 'I-Drug']]
        """
        enbs1, enbe1 = mappings[enid1]
        en1 = nnsents[enbs1: enbe1+1]
        si1, sii1, fs1, ors1 = format_relen(en1, 1, nsents)
        enbs2, enbe2 = mappings[enid2]
        en2 = nnsents[enbs2: enbe2+1]
        si2, sii2, fs2, ors2 = format_relen(en2, 2, nsents)
        sent_diff = abs(si1 - si2)
        
        en1t = en1[0][-1].split("-")[-1]
        en2t = en2[0][-1].split("-")[-1]

        true_pairs.add((enid1, enid2))
        
        if (en1t, en2t) not in valid_comb:
            continue
        
        if sent_diff <= CUTOFF:
            check_tags(fs1, fs2)
            assert (en1t, en2t) in valid_comb, f"{en1t} {en2t}"
            if DO_BIN:
                pos_samples.append((sent_diff, "pos", fs1, fs2, en1t, en2t, enid1, enid2, fid))
            else:
                pos_samples.append((sent_diff, rel_type, fs1, fs2, en1t, en2t, enid1, enid2, fid))

    return pos_samples, true_pairs
        

def gene_neg_relation(perm_pairs, true_pairs, mappings, ens, e2i, nnsents, nsents, valid_comb, fid=None):
    neg_samples = []
    for each in perm_pairs:
        enid1, enid2 = each
        
        # not in true relation
        if (enid1, enid2) in true_pairs:
            continue
        
        enc1 = ens[e2i[enid1]]
        enc2 = ens[e2i[enid2]]

        enbs1, enbe1 = mappings[enid1]
        en1 = nnsents[enbs1: enbe1+1]
        si1, sii1, fs1, ors1 = format_relen(en1, 1, nsents)
        enbs2, enbe2 = mappings[enid2]
        en2 = nnsents[enbs2: enbe2+1]
        si2, sii2, fs2, ors2 = format_relen(en2, 2, nsents)
        sent_diff = abs(si1 - si2)
        
        en1t = en1[0][-1].split("-")[-1]
        en2t = en2[0][-1].split("-")[-1]
        
        if (en1t, en2t) not in valid_comb:
            continue
        
        if sent_diff <= CUTOFF:
            check_tags(fs1, fs2)
            assert (en1t, en2t) in valid_comb, f"{en1t} {en2t}"
            if fid:
                neg_samples.append((sent_diff, NEG_REL, fs1, fs2, en1t, en2t, enid1, enid2, fid))
            else:
                neg_samples.append((sent_diff, NEG_REL, fs1, fs2, en1t, en2t, enid1, enid2))
    
    return neg_samples

    
def create_training_samples(file_path, valids=None, valid_comb=None):
    fids = []
    root = Path(file_path)
    
    dpos = defaultdict(list)
    dneg = defaultdict(list)
    
    for txt_fn in root.glob("*.txt"):
        fids.append(txt_fn.stem)
        ann_fn = root / (txt_fn.stem+".ann")

        # load text
        txt = load_text(txt_fn)
        pre_txt, sents = pre_processing(txt_fn, deid_pattern=MIMICIII_PATTERN)
        e2i, ens, rels = read_annotation_brat(ann_fn)
        i2e = {v: k for k, v in e2i.items()}
        
        nsents, sent_bound = generate_BIO(sents, ens, file_id="", no_overlap=False, record_pos=True)
        print(nsents)
        total_len = len(nsents)
        nnsents = [w for sent in nsents for w in sent]
        mappings = create_entity_to_sent_mapping(nnsents, ens, i2e)

        pos_samples, true_pairs = gene_true_relations(
            rels, mappings, ens, e2i, nnsents, nsents, valid_comb, fid=txt_fn.stem)
        perm_pairs = get_permutated_relation_pairs(e2i)
        neg_samples = gene_neg_relation(
            perm_pairs, true_pairs, mappings, ens, e2i, nnsents, nsents, valid_comb, fid=txt_fn.stem)
        
        for pos_sample in pos_samples:
            dpos[pos_sample[0]].append(pos_sample)
        for neg_sample in neg_samples:
            dneg[neg_sample[0]].append(neg_sample)
        
    return dpos, dneg


def create_test_samples(file_path, valids=None, valid_comb=None):
    #create a separate mapping file
    rel_mappings = []
    #
    fids = []
    root = Path(file_path)
    preds = defaultdict(list)
    
    for txt_fn in root.glob("*.txt"):
        fids.append(txt_fn.stem)
        ann_fn = root / (txt_fn.stem + ".ann")

        # load text
        txt = load_text(txt_fn)
        pre_txt, sents = pre_processing(txt_fn, deid_pattern=MIMICIII_PATTERN)
        e2i, ens, _ = read_annotation_brat(ann_fn)
        i2e = {v: k for k, v in e2i.items()}
        
        nsents, sent_bound = generate_BIO(sents, ens, file_id="", no_overlap=False, record_pos=True)
        total_len = len(nsents)
        nnsents = [w for sent in nsents for w in sent]
        mappings = create_entity_to_sent_mapping(nnsents, ens, i2e)
        
        perm_pairs = get_permutated_relation_pairs(e2i)
        pred = gene_neg_relation(perm_pairs, set(), mappings, ens, e2i, nnsents, nsents, valid_comb, fid=txt_fn.stem)
        for idx, pred_s in enumerate(pred):
            preds[pred_s[0]].append(pred_s)
            
    return preds

In [ ]:
def en_sent_id(en_pos, send_bound):
    e_s = en_pos[0]
    e_e = en_pos[1]
    for k, v in sent_bound.items():
        s_s = v[0]
        s_e = v[1]
        if e_s >= s_s and e_s <= s_e and e_e >s_e :
            print("entity is in two sentence")
        if e_s >= s_s and e_s <= s_e:
            return k
        

def extract_entity_comb_for_relation(e2idx, entities, rels, sent_bound):
    #'T1': 0
    #'meropenem', 'Drug', (4534, 4543)
    #('Strength-Drug', 'T5', 'T39')
    rn = defaultdict(list)
    rl = []
    for rel in rels:
        rtype = rel[0]
        en1 = rel[1]
        en2 = rel[2]
        en1_type = entities[e2idx[en1]][1]
        en2_type = entities[e2idx[en2]][1]
        rn[rtype].append((en1_type, en2_type))
        en1_pos = entities[e2idx[en1]][2]
        e1_n = en_sent_id(en1_pos, sent_bound)
        en2_pos = entities[e2idx[en2]][2]
        e2_n = en_sent_id(en2_pos, sent_bound)
        rl.append(abs(e1_n-e2_n))
    return rn, rl

In [ ]:
"""
Output strategy:

1. by cross-distance
- no cross distance; all in one
- by cross distance; all in unique
- by partial cross distance; within-sentence vs. cross sentence

2. relation format
- [CLS] S1 [SEP] S2 [SEP]
- [CLS] S1 S2 [SEP]

We only handle (1) in data generation here, (2) will be handled by the data_utils
"""
def to_tsv(data, fn):
    header = "\t".join([str(i+1) for i in range(len(data[0]))])
    with open(fn, "w") as f:
        f.write(f"{header}\n")
        for each in data:
            d = "\t".join([str(e) for e in each])
            f.write(f"{d}\n")


def to_5_cv(data, ofd):
    if not os.path.isdir(ofd):
        os.mkdir(ofd)
    
    np.random.seed(13)
    np.random.shuffle(data)
    
    dfs = np.array_split(data, 5)
    a = [0,1,2,3,4]
    for each in combinations(a, 4):
        b = list(set(a) - set(each))[0]
        n = dfs[b]
        m = []
        for k in each:
            m.extend(dfs[k])
        if not os.path.isdir(os.path.join(ofd, f"sample{b}")):
            os.mkdir(os.path.join(ofd, f"sample{b}"))
        
        to_tsv(m, os.path.join(ofd, f"sample{b}", "train.tsv"))
        to_tsv(n, os.path.join(ofd, f"sample{b}", "dev.tsv"))


def all_in_one(*dd, dn="2018n2c2", do_train=True):
    data = []
    for d in dd:
        for k, v in d.items():
            for each in v:
                data.append(each[1:])
    
    output_path = f"../data/{dn}_aio_th{CUTOFF}"
    p = Path(output_path)
    p.mkdir(parents=True, exist_ok=True)
    
    if do_train:
        to_tsv(data, p/"train.tsv")
        if OUTPUT_CV:
            to_5_cv(data, p.as_posix())
    else:
        to_tsv(data, p/"test.tsv")
    

def all_in_unique(*dd, dn="2018n2c2", do_train=True):
    for idx in range(CUTOFF+1):
        data = []
        for d in dd:
            for k, v in d.items():
                for each in v:
                    if k == idx:
                        data.append(each[1:])
        
        output_path = f"../data/{dn}_aiu_th{CUTOFF}"
        p = Path(output_path) / f"cutoff_{idx}"
        p.mkdir(parents=True, exist_ok=True)
        if do_train:
            to_tsv(data, p/"train.tsv")
            if OUTPUT_CV:
                to_5_cv(data, p.as_posix())
        else:
            to_tsv(data, p/"test.tsv")

## GLOBAL CONFIG VARIABLES

In [ ]:
"""
We did not automated this process
You can manually create these variables
"""

'\nWe did not automated this process\nYou can manually create these variables\n'

In [ ]:
# general pre-defined special tags
EN1_START = "[s1]"
EN1_END = "[e1]"
EN2_START = "[s2]"
EN2_END = "[e2]"
NEG_REL = "NonRel"
# max valid cross sentence distance
CUTOFF = 1
# output 5-fold cross validation data
OUTPUT_CV = False
# do binary classification (if false, then we do multiclass classification)
DO_BIN = False

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
n2c2_training = "/content/drive/MyDrive/chia/chia_with_scope/"
n2c2_test = "/content/drive/MyDrive/test_data_Tasks1&3/"

In [ ]:
def to_BIO_tsv(data, fn):
    #header = "\t".join([str(i+1) for i in range(len(data[0]))])
    with open(fn, "w") as f:
        #f.write(f"{header}\n")
        for each in data:
            for ea in each:
              d = "\t".join([str(e) for e in [ea[0], ea[3]]])
              print(d)
              f.write(f"{d}\n")
            

In [ ]:
## get training stats
file_ids = [fn.stem for fn in Path(n2c2_training).glob("*.ann")]
train_length = round(len(file_ids)*0.74)
dev_length = train_length + round(len(file_ids)*0.13)

print(file_ids[:train_length])
print("total training files: ", len(file_ids))

relation_types = set()
ann_rel_comb = defaultdict(list)
rel_sent_dis = []
with open('/content/drive/MyDrive/BioMegatron/ChiaNER/DATA_DIR/NER/train.tsv', "w") as f1:
  for each_file in file_ids[:train_length]:
      input_file = n2c2_training + "/" + each_file + ".txt"
      ann_file = n2c2_training + "/" +  each_file + ".ann"
      txt, sents = pre_processing(input_file, MIMICIII_PATTERN)
      e2idx, entities, rels = read_annotation_brat(ann_file)
      nsents, sent_bound = generate_BIO(sents, entities)
      for each in nsents:
            for ea in each:
              d = "\t".join([str(e) for e in [ea[0], ea[3]]])
              f1.write(f"{d}\n")
              if ea[0] in set(['.','?','!',';']) :
                f1.write("\n")
      f1.write("\n") 

with open('/content/drive/MyDrive/BioMegatron/ChiaNER/DATA_DIR/NER/dev.tsv', "w") as f2:
  for each_file in file_ids[train_length:dev_length]:
      input_file = n2c2_training + "/" + each_file + ".txt"
      ann_file = n2c2_training + "/" +  each_file + ".ann"
      txt, sents = pre_processing(input_file, MIMICIII_PATTERN)
      e2idx, entities, rels = read_annotation_brat(ann_file)
      nsents, sent_bound = generate_BIO(sents, entities)
      for each in nsents:
            for ea in each:
              d = "\t".join([str(e) for e in [ea[0], ea[3]]])
              f2.write(f"{d}\n")
              if ea[0] in set(['.','?','!',';']) :
                f2.write("\n")
      f2.write("\n") 

with open('/content/drive/MyDrive/BioMegatron/ChiaNER/DATA_DIR/NER/test.tsv', "w") as f3:
  for each_file in file_ids[dev_length:]:
      input_file = n2c2_training + "/" + each_file + ".txt"
      ann_file = n2c2_training + "/" +  each_file + ".ann"
      txt, sents = pre_processing(input_file, MIMICIII_PATTERN)
      e2idx, entities, rels = read_annotation_brat(ann_file)
      nsents, sent_bound = generate_BIO(sents, entities)
      for each in nsents:
            for ea in each:
              d = "\t".join([str(e) for e in [ea[0], ea[3]]])
              f3.write(f"{d}\n") 
              if ea[0] in set(['.','?','!',';']) :
                f3.write("\n")    
      f3.write("\n") 
    # er_comb, er_n = extract_entity_comb_for_relation(e2idx, entities, rels, sent_bound)
    # rel_sent_dis.extend(er_n)
    # for k, v in er_comb.items():
    #     for vv in v:
    #         ann_rel_comb[k].append(vv)
    
    # for each in rels:
    #     relation_types.add(each[0])

['NCT01497639_inc', 'NCT02970773_inc', 'NCT02675153_exc', 'NCT03066440_exc', 'NCT02918409_inc', 'NCT02430740_inc', 'NCT01000155_exc', 'NCT01809041_inc', 'NCT02375295_exc', 'NCT02715518_exc', 'NCT01579604_exc', 'NCT02364648_exc', 'NCT03140423_inc', 'NCT02804126_inc', 'NCT02537899_exc', 'NCT02455921_inc', 'NCT01895946_inc', 'NCT02566928_exc', 'NCT01205334_inc', 'NCT02985242_exc', 'NCT02773173_inc', 'NCT02330757_inc', 'NCT02807857_inc', 'NCT03045562_exc', 'NCT02466113_exc', 'NCT01912651_exc', 'NCT02462317_exc', 'NCT02957877_exc', 'NCT03373318_inc', 'NCT02961582_inc', 'NCT01373684_exc', 'NCT02537899_inc', 'NCT01803438_exc', 'NCT02735902_exc', 'NCT01816997_inc', 'NCT03234816_exc', 'NCT03228017_inc', 'NCT02283905_exc', 'NCT03177811_exc', 'NCT00440245_exc', 'NCT02390973_inc', 'NCT00324363_exc', 'NCT02827487_inc', 'NCT02951520_inc', 'NCT01314898_exc', 'NCT02478346_inc', 'NCT02426034_inc', 'NCT00182520_exc', 'NCT03064867_exc', 'NCT02102243_inc', 'NCT03195153_inc', 'NCT03119766_exc', 'NCT0213736